In [1]:
import httpx
import asyncio
import datetime as dt
import pylint
from lxml import etree
import time
import re

In [2]:
OLDEST = dt.datetime(1958, 8, 4)

In [3]:
def to_saturday(date, scalar=1):
    # rounds the date to a Saturday
    while date.weekday() != 5:
        date = date + scalar*dt.timedelta(days=1)
    return date

In [4]:
def date_generator(latest_date, delta=1):
    # infintite generator for dates that takes the timedelta as a paramater
    latest_date = to_saturday(latest_date)
    curr = latest_date
    while True:
        yield curr
        curr -= dt.timedelta(weeks=delta)

In [ ]:
dates = date_generator(round_date(dt.datetime.today()))
tasks = [await single_page(client, date) for date in dates]

pages = await asyncio.gather(*tasks)



NameError: name 'client' is not defined

In [5]:
# testing parser feed() and read_events() methods
async def single_page(client, date):
    url = get_url(date)
    parser = etree.HTMLParser(target=BillboardTarget())
    main_frame = []
    async with client.stream('GET', URL) as response:
        async for chunk in response.aiter_text():
            parser.feed(chunk)
            for content in parser.read_events():
                main_frame += incrementally_build_frame(content)
    return date, main_frame
                

In [6]:
class BillboardTarget:
    VALID_TAG = 'ul'
    VALID_CLASS = 'o-chart-results-list-row //'
    def __init__(self):
        self.content = []
        self.capture = False
        self.curr_tag = None
        self.curr_data = ''

    def start(self, tag, attrib):
        if tag == self.VALID_TAG and self.VALID_CLASS in attrib['class']:
            self.capture = True
            self.curr_tag = tag
            self.curr_data = ''

    def end(self, tag):
        if self.capture and tag == self.curr_tag:
            self.content.append(self.curr_data)
            self.capture = False
            self.curr_tag = None

    def data(self, text):
        if self.capture and not text.isspace():
            text = re.sub(r"NEW", "", text)
            text = re.sub(r"RE-\nENTRY", "", text)
            self.curr_data += text.strip() + '_'

    def close(self):
        return self.content


In [55]:
test_doc = open('Billboard-page.html')
parser = etree.HTMLParser(target=BillboardTarget())
e = etree.parse(test_doc, parser)
test_doc.close()
e

['1_Roses Are Red (My Love)_Bobby Vinton_1_1_7_1_1_7_',
 '2_The Wah Watusi_The Orlons_4_2_7_4_2_7_',
 "3_I Can_'_t Stop Loving You_Ray Charles_3_1_12_3_1_12_",
 '4_The Stripper_David Rose and His Orchestra_2_1_11_2_1_11_',
 '5_Sealed With A Kiss_Brian Hyland_5_5_7_5_5_7_',
 '6_Wolverton Mountain_Claude King_7_6_9_7_6_9_',
 '7_Johnny Get Angry_Joanie Sommers_11_7_9_11_7_9_',
 '8_Speedy Gonzales_Pat Boone_13_8_6_13_8_6_',
 '9_Gravy (For My Mashed Potatoes)_Dee Dee Sharp_9_9_6_9_9_6_',
 '10_Palisades Park_Freddy Cannon_6_3_11_6_3_11_',
 "11_Al Di La_'_Emilio Pericoli_10_6_10_10_6_10_",
 '12_Ahab, The Arab_Ray Stevens_18_12_4_18_12_4_',
 '13_Breaking Up Is Hard To Do_Neil Sedaka_19_13_4_19_13_4_',
 "14_It Keeps Right On A-Hurtin_'_Johnny Tillotson_8_3_11_8_3_11_",
 "15_Dancin_'_Party_Chubby Checker_20_15_5_20_15_5_",
 "16_I_'_ll Never Dance Again_Bobby Rydell_14_14_8_14_14_8_",
 '17_Having A Party_Sam Cooke_17_17_9_17_17_9_',
 '18_Theme From Dr. Kildare (Three Stars Will Shine Tonight)_Ric

In [ ]:
def incrementally_build_frame(e):
    subs = (
        (re.compile(r"_\'_"), "'"), 
        (re.compile(r"_\"_"), " \" "), 
        (re.compile(r"\'(?=[A-Z])"), "\'~")
        )
    for pattern, replacement in subs:
        e = [re.sub(pattern, replacement, row) for row in e]
    data = []
    for row in e:
        clean_row = [row.split('_')[i] for i in range(len(row.split('_'))) if row.split('_')[i]]
        data.append([clean_row[i] for i in (0, 1, 2, 5)])
    return data

[['1', 'Roses Are Red (My Love)', 'Bobby Vinton', '7'],
 ['2', 'The Wah Watusi', 'The Orlons', '7'],
 ['3', "I Can't Stop Loving You", 'Ray Charles', '12'],
 ['4', 'The Stripper', 'David Rose and His Orchestra', '11'],
 ['5', 'Sealed With A Kiss', 'Brian Hyland', '7'],
 ['6', 'Wolverton Mountain', 'Claude King', '9'],
 ['7', 'Johnny Get Angry', 'Joanie Sommers', '9'],
 ['8', 'Speedy Gonzales', 'Pat Boone', '6'],
 ['9', 'Gravy (For My Mashed Potatoes)', 'Dee Dee Sharp', '6'],
 ['10', 'Palisades Park', 'Freddy Cannon', '11'],
 ['11', "Al Di La'~Emilio Pericoli", '10', '10'],
 ['12', 'Ahab, The Arab', 'Ray Stevens', '4'],
 ['13', 'Breaking Up Is Hard To Do', 'Neil Sedaka', '4'],
 ['14', "It Keeps Right On A-Hurtin'~Johnny Tillotson", '8', '8'],
 ['15', "Dancin'~Party", 'Chubby Checker', '5'],
 ['16', "I'll Never Dance Again", 'Bobby Rydell', '8'],
 ['17', 'Having A Party', 'Sam Cooke', '9'],
 ['18',
  'Theme From Dr. Kildare (Three Stars Will Shine Tonight)',
  'Richard Chamberlain',
  '8